<a href="https://colab.research.google.com/github/HajarahM/Mistral-7B/blob/main/Chat_with_MultiplePDFs_Mistral_7B_Instruct1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install langchain
%pip install torch
%pip install sentence_transformers
%pip install faiss-cpu
%pip install huggingface-hub
%pip install pypdf
%pip -q install accelerate
%pip install llama-cpp-python
%pip -q install git+https://github.com/huggingface/transformers


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.chains import RetrievalQA
from langchain.embeddings import OllamaEmbeddings
from langchain.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader

In [4]:
#load pdf files
loader = DirectoryLoader("./data1/")
data = loader.load()

In [5]:
print(data)

[Document(page_content='CHAPTER 4\n\nTHE ARBITRATION AND CONCILIATION ACT.\n\nArrangement of Sections.\n\nSection\n\nPART I—PRELIMINARY.\n\n1. 2.\n\nApplication. Interpretation.\n\nPART II—ARBITRATION.\n\n3. 4. Waiver of right to object. Stay of legal proceedings. 5. Interim measures by the court. 6. Death of a party. 7. Receipt of written communications. 8. Extent of court intervention. 9. 10. Determination of number of arbitrators. 11. Appointment of arbitrators. 12. Grounds for challenge. 13. Challenge procedure. 14. Failure or impossibility to act. 15. Termination of mandate and substitution of arbitrator. 16. Competence of arbitral tribunal to rule on its jurisdiction. 17. Power of arbitral tribunal. 18. Equal treatment of parties. 19. Determination of rules of procedure. 20. Place of arbitration. 21. Commencement of arbitral proceedings. 22. Language. 23. Statements of claim and defence. 24. Hearing and written submissions. 25. Default of a party. 26. Experts. 27. Court assistanc

In [6]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10, separators=['\n\n', '\n', '.'])

text_chunks = text_splitter.split_documents(data)


In [7]:
len(text_chunks)

177

In [29]:
#get the third chunk
text_chunks[3]

Document(page_content='Form of arbitration agreement.\n\n28. Rules applicable to substance of dispute. 29. Decision making by panel of arbitrators. 30. Settlement. 31. Form and contents of arbitral award. 32. Termination of arbitral proceedings. 33. Correction and interpretation of arbitral award; additional award. 34. Application for setting aside arbitral award. 35. Recognition and enforcement of award. 36. Enforcement. 37. Bankruptcy. 38. Questions of law arising in domestic arbitration.', metadata={'source': 'data1/Arbitration Act.pdf'})

In [10]:
#Step 06:Downlaod the Embeddings
embeddings = OllamaEmbeddings(model="mistral")

In [11]:
from langchain.vectorstores import Chroma
vector_store = Chroma.from_documents(text_chunks, embedding=embeddings)
print('saved embeddings to vector_store')

saved embeddings to vector_store


In [12]:
#Step 08: Create Embeddings for each of the Text Chunk
import os
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
print('saving embeddings to vector_store')
folder_path="FAISS_vector_store"
if os.path.exists(folder_path):
    faiss_index=FAISS.load_local(folder_path, embeddings)
    faiss_index.merge_from(vector_store)
    faiss_index.save_local(folder_path)
else:
    vector_store.save_local(folder_path)

KeyboardInterrupt: 

In [30]:
#Import Model
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = Ollama(
    base_url="http://localhost:11434",
    model = "mistral",
    verbose=True,
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    )

In [31]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 1}))

In [34]:
query = "how do you enforce an arbitration award?"

In [35]:
qa.run(query)


To enforce an arbitration award, the party that was awarded damages can take several steps. First, they can try to negotiate with the other party to recover the amount owed. If negotiations fail, they can file a writ of execution in court, which allows them to seize and sell assets owned by the other party to pay off the arbitration award.

If the assets seized are insufficient to cover the award, the party can file a lien on other assets owned by the other party. This gives them the right to take possession of and sell other assets until the award is paid in full. If the other party refuses to pay the award, the party can also seek legal action, such as filing a lawsuit or pursuing criminal charges.

'\nTo enforce an arbitration award, the party that was awarded damages can take several steps. First, they can try to negotiate with the other party to recover the amount owed. If negotiations fail, they can file a writ of execution in court, which allows them to seize and sell assets owned by the other party to pay off the arbitration award.\n\nIf the assets seized are insufficient to cover the award, the party can file a lien on other assets owned by the other party. This gives them the right to take possession of and sell other assets until the award is paid in full. If the other party refuses to pay the award, the party can also seek legal action, such as filing a lawsuit or pursuing criminal charges.'

In [32]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")


Under Section 28 of the Arbitration Act, the rules applicable to the substance of the dispute would still be in effect. The decision-making process by the panel of arbitrators would also continue, but if a party dies during the proceedings, their rights and obligations under the arbitration agreement would pass on to their estate.

If the award is already made before the death of a party, it would still be binding on the parties involved. However, in some cases, the deceased party's estate may challenge the award or seek a settlement if there are sufficient funds available.

If the award has not yet been made when a party dies, the arbitration proceedings would likely continue, but with the representation of the deceased party's estate. The panel of arbitrators would need to consider any relevant legal issues arising from the death of the party, such as whether the deceased party had previously waived their right to challenge the award or whether the arbitration agreement provides for

SystemExit: 

/opt/miniconda3/envs/data_processor/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
